In [1]:
from pyscf import gto  # PySCF is a quantum chemistry python module
import SCF
import numpy as np

/opt/anaconda3/envs/venv/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# Input Data
mol_h2o = gto.M(unit="Bohr",
                atom="O 0.000000000000  -0.143225816552   0.000000000000;"
                + "H 1.638036840407   1.136548822547  -0.000000000000;"
                + "H -1.638036840407   1.136548822547  -0.000000000000",
                basis='STO-3g')
mol_h2o.build()

# Convergence Criteria
E_conv_threshold = 1.0E-10
D_conv_threshold = 1.0E-8
max_iterations = 1000

# get the integrals
Suv = mol_h2o.intor('int1e_ovlp')  # Overlap Integrals
Tuv = mol_h2o.intor('int1e_kin')  # Kinetic Energy 1 electron integrals
Vuv = mol_h2o.intor('int1e_nuc')  # Nuclear Repulsion 1 electron integrals
eri = mol_h2o.intor("int2e")  # Electron Repulsion 2 electron integrals

In [3]:
print(Tuv.shape)
print(Vuv.shape)

print(eri.shape) # 4-D, electron1, electron2, spin1, spin2

(7, 7)
(7, 7)
(7, 7, 7, 7)


In [4]:
Enuc = SCF.calc_nuclear_repulsion_energy(mol_h2o)
print(Enuc)

8.00236706181077


In [5]:
Huv = SCF.calc_hcore_matrix(Tuv, Vuv)
print(Huv[0, 0], Huv[3, 4], Huv[4, 3])
print(Huv.shape)

-32.57739541261037 0.0 0.0
(7, 7)


In [6]:
Duv = SCF.calc_initial_density(mol_h2o)
print(Duv)
print(Duv.shape)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
(7, 7)


In [7]:
for it in range(max_iterations):
    Fuv = SCF.calc_fock_matrix(mol_h2o, Huv, eri, Duv)
    mo_e, mo_c = SCF.solve_Roothan_equations(Fuv, Suv)
    
    Etot_new = SCF.calc_tot_energy(Fuv, Huv, Duv, Enuc)
    Duv_new = SCF.form_density_matrix(mol_h2o, mo_c)
    break

In [8]:
print(Fuv[0, 0])
print(Fuv[2, 5])
print(Fuv[5, 2])

-32.57739541261037
-1.6751501447185013
-1.6751501447185015


In [9]:
print(mo_e)
print()
print(mo_c[0, :])
print(mo_e.shape, mo_c.shape)

[-32.57830292  -8.08153571  -7.55008599  -7.36396923  -7.34714487
  -4.00229867  -3.98111115]

[-1.00154358e+00 -2.33624458e-01  3.49050635e-16 -8.56842145e-02
 -1.10314884e-29  4.82226067e-02 -1.11404715e-15]
(7,) (7, 7)


In [10]:
print(Etot_new)

8.00236706181077


In [11]:
print(Duv_new[0, 0])
print(Duv_new[2, 5], Duv_new[5, 2])

2.130023428655503
-0.292263302096531 -0.292263302096531
